In [1]:
from sklearnex import patch_sklearn

patch_sklearn()

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import TruncatedSVD,LatentDirichletAllocation
from sklearn.neighbors import NearestNeighbors
import Levenshtein
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from sklearn import preprocessing
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from IPython.display import display
import collections
from tqdm import tqdm
import string
import Levenshtein
import difflib
import unidecode
import pickle

tqdm.pandas()



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
%load_ext Cython


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


/kaggle/input/foursquare-location-matching/sample_submission.csv
/kaggle/input/foursquare-location-matching/pairs.csv
/kaggle/input/foursquare-location-matching/train.csv
/kaggle/input/foursquare-location-matching/test.csv


**References:**

https://www.kaggle.com/code/ryotayoshinobu/foursquare-lightgbm-baseline

https://www.kaggle.com/code/guoyonfan/binary-lgb-baseline-0-834


In [2]:
df_train=pd.read_csv('../input/foursquare-location-matching/train.csv')



In [3]:
df_train.columns

Index(['id', 'name', 'latitude', 'longitude', 'address', 'city', 'state',
       'zip', 'country', 'url', 'phone', 'categories', 'point_of_interest'],
      dtype='object')

In [4]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        #else:
        #    df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [5]:
count_vectorizer = CountVectorizer(max_features=10,strip_accents="unicode")
count_vectorizer = count_vectorizer.fit(df_train["categories"].dropna())


def get_tsvd_transformers(data,variable,n_components):
    
    variable_vectorizer = TfidfVectorizer(strip_accents="unicode")
    variable_vectorizer = variable_vectorizer.fit(data[variable].fillna(""))
    variable_vectors=variable_vectorizer.transform(data[variable].fillna(""))

    tsvd_transformer=TruncatedSVD(n_components=n_components)
    tsvd_transformer=tsvd_transformer.fit(variable_vectors)
    #tsvd_vectors=tsvd_tranformer.transform(variable_vectors)
    
    return variable_vectorizer,tsvd_transformer

def get_lda_transformers(data,variable,n_components):
    
    variable_vectorizer = TfidfVectorizer(strip_accents="unicode")
    variable_vectorizer = variable_vectorizer.fit(data[variable].fillna(""))
    variable_vectors=variable_vectorizer.transform(data[variable].fillna(""))

    lda_transformer=LatentDirichletAllocation(n_components=n_components)
    lda_transformer=lda_transformer.fit(variable_vectors)
    #tsvd_vectors=tsvd_tranformer.transform(variable_vectors)
    
    return variable_vectorizer,lda_transformer

name_vectorizer,name_tsvd_transformer=get_tsvd_transformers(df_train,"name",7)
#name_vectorizer,name_lda_transformer=get_lda_transformers(df_train,"name",7)

categories_vectorizer,categories_tsvd_transformer=get_tsvd_transformers(df_train,"categories",7)
#categories_vectorizer,categories_lda_transformer=get_lda_transformers(df_train,"categories",7)

#address_vectorizer,address_tsvd_transformer=get_tsvd_transformers(df_train,"address",7)


In [6]:
with open('count_vectorizer_categories_foursquare_v10.pkl', 'wb') as f:
    pickle.dump(count_vectorizer, f)
    
with open('variable_vectorizer_name_foursquare_v10.pkl', 'wb') as f:
    pickle.dump(name_vectorizer, f)
    
with open('tsvd_vectorizer_name_foursquare_v10.pkl', 'wb') as f:
    pickle.dump(name_tsvd_transformer, f)
    
with open('variable_vectorizer_categories_foursquare_v10.pkl', 'wb') as f:
    pickle.dump(categories_vectorizer, f)
    
with open('tsvd_vectorizer_categories_foursquare_v10.pkl', 'wb') as f:
    pickle.dump(categories_tsvd_transformer, f)
    

    
    

In [7]:
%%cython
import numpy as np  # noqa
cpdef int FastLCS(str S, str T):
    cdef int i, j
    cdef int cost
    cdef int v1,v2,v3,v4
    cdef int[:, :] dp = np.zeros((len(S) + 1, len(T) + 1), dtype=np.int32)
    for i in range(len(S)):
        for j in range(len(T)):
            cost = (int)(S[i] == T[j])
            v1 = dp[i, j] + cost
            v2 = dp[i + 1, j]
            v3 = dp[i, j + 1]
            v4 = dp[i + 1, j + 1]
            dp[i + 1, j + 1] = max((v1,v2,v3,v4))
    return dp[len(S)][len(T)]

In [8]:
def process_country(df_data):
    
    list_country=  ['US',
     'TR',
     'ID',
     'JP',
     'TH',
     'RU',
     'BR',
     'MY',
     'BE',
     'GB',
     'PH',
     'MX',
     'SG',
     'KR',
     'DE',
     'FR',
     'ES']
        
    df_data["country_popular"]=df_data["country"].copy()
        
    df_data.loc[~df_data["country"].isin(list_country),"country_popular"]="OTHER"
        
        
    return df_data


def create_n_words(data):

    data.loc[data['name'].notnull(),'n_words_name']=data.loc[data['name'].notnull(),'name'].apply(lambda x: len(str(x).split()))

    data.loc[data['name'].notnull(),'n_characters_name']=data.loc[data['name'].notnull(),'name'].apply(lambda x: len(str(x).replace(" ","")))
    
    data.loc[data['name_match_id'].notnull(),'n_words_name_match_id']=data.loc[data['name_match_id'].notnull(),'name_match_id'].apply(lambda x: len(str(x).split()))
    
    data.loc[data['name_match_id'].notnull(),'n_characters_name_match_id']=data.loc[data['name_match_id'].notnull(),'name_match_id'].apply(lambda x: len(str(x).replace(" ","")))
    
    data.loc[data['address'].notnull(),'n_words_address']=data.loc[data['address'].notnull(),'address'].apply(lambda x: len(str(x).split()))
    
    data.loc[data['address'].notnull(),'n_characters_address']=data.loc[data['address'].notnull(),'address'].apply(lambda x: len(str(x).replace(" ","")))
    
    data.loc[data['address_match_id'].notnull(),'n_words_address_match_id']=data.loc[data['address_match_id'].notnull(),'address_match_id'].apply(lambda x: len(str(x).split()))
    
    data.loc[data['address_match_id'].notnull(),'n_characters_address_match_id']=data.loc[data['address_match_id'].notnull(),'address_match_id'].apply(lambda x: len(str(x).replace(" ","")))
    
    data.loc[data['categories'].notnull(),'n_words_categories']=data.loc[data['categories'].notnull(),'categories'].apply(lambda x: len(str(x).split()))
    
    data.loc[data['categories_match_id'].notnull(),'n_words_categories_match_id']=data.loc[data['categories_match_id'].notnull(),'categories_match_id'].apply(lambda x: len(str(x).split()))
    
    return data

    
def create_equal_indicator_features(data,equal_variables):
    
    for name_variable in equal_variables:
        
        name_match_variable=f'{name_variable}_match_id'
        name_equal_indicator=f'equal_{name_variable}'
        
        data[name_equal_indicator]=(data[name_variable]==data[name_match_variable]).astype('int8')
        
        condition=(data[name_variable]==np.nan)|(data[name_match_variable]==np.nan)
        
        data.loc[condition,name_equal_indicator]=np.nan
        
        print(name_variable)
        
    return data



def jaccard_distance(id_str,id_match_str):

    try :
        score = len((id_str & id_match_str)) / len((id_str | id_match_str))
        
    except :
        
        score=np.nan
    
    return score

def first_indicator(id_str,id_match_str):

    try :
        score = int(id_str.split()[0] in id_match_str)
        
    except :
        
        score=np.nan
    
    return score

def last_indicator(id_str,id_match_str):

    try :
        score = int(id_str.split()[-1] in id_match_str)
        
    except :
        
        score=np.nan
    
    return score

def first_match_indicator(id_str,id_match_str):

    try :
        score = len((id_str & id_match_str)) / len(id_str)
        
    except :
        
        score=np.nan
    
    return score

def second_match_indicator(id_str,id_match_str):

    try :
        score = len((id_str & id_match_str)) / len(id_match_str)
        
    except :
        
        score=np.nan
    
    return score



def levenshtein_distance(id_str,id_match_str):
    
    try :
    
        score=Levenshtein.distance(id_str, id_match_str)
        
    except :
        
        score=np.nan
    
    return score

def jaro_winkler(id_str,id_match_str):
    
    try :
    
        score=Levenshtein.jaro_winkler(id_str, id_match_str)
        
    except :
        
        score=np.nan
    
    return score

def sequence_matcher(id_str,id_match_str):
    
    try :
    
        score=difflib.SequenceMatcher(None, id_str,id_match_str).ratio()
        
    except :
        
        score=np.nan
    
    return score


def metrics_similiarity(data,name_variable,name_match_variable=None,distances_list=["jaccard"],suffix='',make_unidecode=0,remove_vowels=0):
    ## check if there should be separation between words for some punctuations
    
    scores_jaccard = []
    scores_levenshtein=[]
    scores_jaro_winkler=[]
    scores_sequence_matcher=[]
    scores_lcs=[]
    scores_first_match_indicator=[]
    scores_second_match_indicator=[]
    scores_first_indicator=[]
    scores_last_indicator=[]
    str_vowels="aeiou"
    
    
    if name_match_variable is None :
    
        name_match_variable=f'{name_variable}_match_id'
    
    for id_str, id_match_str in tqdm(zip(data[name_variable].fillna("nullvalue").to_numpy(), data[name_match_variable].fillna("nullvalue_match").to_numpy())):
    
        if make_unidecode==1 :
            id_str=unidecode.unidecode(id_str)
            id_match_str=unidecode.unidecode(id_match_str)
            
        id_str=id_str.lower().translate(str.maketrans(string.punctuation,' '*len(string.punctuation)))
        if remove_vowels==1:
            id_str=id_str.lower().translate(str.maketrans('', '',str_vowels))
        id_str=" ".join(id_str.split()) # this is new
            
        id_match_str=id_match_str.lower().translate(str.maketrans(string.punctuation,' '*len(string.punctuation)))
        if remove_vowels==1:
            id_match_str=id_match_str.lower().translate(str.maketrans('', '',str_vowels))
        id_match_str=" ".join(id_match_str.split())

        
        id_str_set=set(id_str.split())
        id_match_str_set=set(id_match_str.split())
        
                
        if "jaccard" in distances_list :
                
            score_jaccard=jaccard_distance(id_str_set,id_match_str_set)
                
            scores_jaccard.append(score_jaccard)
            
        if "first_match_indicator" in distances_list :
                
            score_first_match_indicator=first_match_indicator(id_str_set,id_match_str_set)
                
            scores_first_match_indicator.append(score_first_match_indicator)
            
        if "second_match_indicator" in distances_list :
                
            score_second_match_indicator=second_match_indicator(id_str_set,id_match_str_set)
                
            scores_second_match_indicator.append(score_second_match_indicator)
            
        if "first_indicator" in distances_list :
                
            score_first_indicator=first_indicator(id_str,id_match_str)
                
            scores_first_indicator.append(score_first_indicator)
            
        if "last_indicator" in distances_list :
                
            score_last_indicator=last_indicator(id_str,id_match_str)
                
            scores_last_indicator.append(score_last_indicator)
            
            
        if "levenshtein" in distances_list:
                
            score_levenshtein=levenshtein_distance(id_str,id_match_str)
                
            scores_levenshtein.append(score_levenshtein)
                
        if "jaro_winkler" in distances_list:
                
            score_jaro_winkler=jaro_winkler(id_str,id_match_str)
                
            scores_jaro_winkler.append(score_jaro_winkler)
            
        if "sequence_matcher" in distances_list:
            
            score_sequence_matcher=sequence_matcher(id_str,id_match_str)
                
            scores_sequence_matcher.append(score_sequence_matcher)
            
        if "lcs" in distances_list:
            
            score_lcs=FastLCS(id_str,id_match_str)
                
            scores_lcs.append(score_lcs)
                
     
    condition=data[name_variable].isnull() | data[name_match_variable].isnull()
            
            
    if "jaccard" in distances_list :
                            
        scores_jaccard=np.array(scores_jaccard)
        
        data[f'jaccard_distance_match_{name_variable}_{suffix}']=scores_jaccard
        
        data.loc[condition,f'jaccard_distance_match_{name_variable}_{suffix}']=np.nan
        
    if "first_match_indicator" in distances_list :
        
        scores_first_match_indicator=np.array(scores_first_match_indicator)
        
        data[f'first_match_indicator_match_{name_variable}_{suffix}']=scores_first_match_indicator
        
        data.loc[condition,f'first_match_indicator_match_{name_variable}_{suffix}']=np.nan
        
    if "second_match_indicator" in distances_list :
        
        scores_second_match_indicator=np.array(scores_second_match_indicator)
        
        data[f'second_match_indicator_match_{name_variable}_{suffix}']=scores_second_match_indicator
        
        data.loc[condition,f'second_match_indicator_match_{name_variable}_{suffix}']=np.nan
        
        
    if "first_indicator" in distances_list :
                            
        scores_first_indicator=np.array(scores_first_indicator)
        
        data[f'first_indicator_match_{name_variable}_{suffix}']=scores_first_indicator
        
        data.loc[condition,f'first_indicator_match_{name_variable}_{suffix}']=np.nan
        
    if "last_indicator" in distances_list :
                            
        scores_last_indicator=np.array(scores_last_indicator)
        
        data[f'last_indicator_match_{name_variable}_{suffix}']=scores_last_indicator
        
        data.loc[condition,f'last_indicator_match_{name_variable}_{suffix}']=np.nan
            
            
    if "levenshtein" in distances_list:
                
        scores_levenshtein=np.array(scores_levenshtein)
        
        data[f'levenshtein_distance_match_{name_variable}_{suffix}']=scores_levenshtein
        
        data.loc[condition,f'levenshtein_distance_match_{name_variable}_{suffix}']=np.nan
                
    if "jaro_winkler" in distances_list:
        
        scores_jaro_winkler=np.array(scores_jaro_winkler)
        
        data[f'jaro_winkler_distance_match_{name_variable}_{suffix}']=scores_jaro_winkler
        
        data.loc[condition,f'jaro_winkler_distance_match_{name_variable}_{suffix}']=np.nan
        
    if "sequence_matcher" in distances_list:
        
        scores_sequence_matcher=np.array(scores_sequence_matcher)
        
        data[f'sequence_matcher_distance_match_{name_variable}_{suffix}']=scores_sequence_matcher
        
        data.loc[condition,f'sequence_matcher_distance_match_{name_variable}_{suffix}']=np.nan
        
    if "lcs" in distances_list:
        
        scores_lcs=np.array(scores_lcs)
        
        data[f'lcs_distance_match_{name_variable}_{suffix}']=scores_lcs
        
        data.loc[condition,f'lcs_distance_match_{name_variable}_{suffix}']=np.nan
    
    

    return data


def get_numbers_from_string(data,name_variable):
    
    name_number_variable=f'numbers_{name_variable}'
    
    data[name_number_variable]=data[name_variable].fillna('nullvalue').apply(lambda x: (" ".join([''.join(filter(str.isdigit, string)) for string in x.split()])).strip())
    
    data.loc[data[name_number_variable]=='',name_number_variable]=np.nan
    
    return data

def get_numbers_match(data,name_variable):

    data=get_numbers_from_string(data,name_variable)

    data=get_numbers_from_string(data,f'{name_variable}_match_id')

    data=metrics_similiarity(data,f'numbers_{name_variable}')
    
    return data


def get_categories_indicator(data,variable_name,transformer,n_features=10):
    
    data_result=transformer.transform(data[variable_name].fillna("null"))
    
    data_result=data_result.toarray()
    
    data_result=pd.DataFrame(data_result)
    
    dict_names = {transformer.vocabulary_[k] : k for k in transformer.vocabulary_}

    list_names=[f'{dict_names[x]}_{variable_name}' for x in range(0,n_features) ]
    
    data_result.columns=list_names
    
    data = pd.concat([data, data_result], axis=1)

    
    return data

def get_tsvd_lda_vectors(data,variable_name,vector_transformer,tsvd_transformer):
    
    variable_vectors=vector_transformer.transform(data[variable_name].fillna(""))
    
    data_result=tsvd_transformer.transform(variable_vectors)
    
    n_features=np.shape(data_result)[1]
    
    data_result=pd.DataFrame(data_result)
        
    list_names=[f'{variable_name}_component_{x}' for x in range(0,n_features) ]
    
    data_result.columns=list_names
    
    data = pd.concat([data, data_result], axis=1)

    return data

In [9]:
def nearest_neighbors(data,n_nearest):
    
    knn = NearestNeighbors(n_neighbors = n_nearest,metric='haversine')
    knn.fit(data, data.index)
    distances,nearest_ids = knn.kneighbors(data,return_distance = True)
    
    return distances,nearest_ids


# based on this https://www.kaggle.com/competitions/coleridgeinitiative-show-us-the-data/discussion/228814#1253355
def jaccard_similarity(row,variable):
    
    try : 
        l1 = row[variable].split(" ")
        l2 = row[f'{variable}_match_id'].split(" ")    
        intersection = len([s for s in l1 if s in l2])
        union = (len(l1) + len(l2)) - intersection
        return float(intersection) / union
    
    except :
        return np.nan
    
    
def jaccard_similarity_like(data,name_variable):
    
    name_match_variable=f'{name_variable}_match_id'
    
    count_vectorizer = CountVectorizer(binary=True,stop_words=['nullvalue'],token_pattern=r"(?u)\b\w+\b",strip_accents="unicode")
    data_result = count_vectorizer.fit_transform(data[name_variable].fillna('nullvalue'))
    dict_id_index = dict(zip(data[name_variable].values, data.index))
    
    index_match_id=[dict_id_index[element] for element in data[name_match_variable]]
    
    distance_vector=data_result.multiply(data_result[index_match_id]).sum(axis=1).A.ravel()
    
    return distance_vector


def filter_candidates(data,n_neighbor):
    
    if n_neighbor > 1 :
    
        data=data.loc[(data['jaccard_distance_match_name_']>0)|(data['jaro_winkler_distance_match_name_']>0.7)|(data['jaccard_distance_match_name_unidecode']>0)|(data['jaro_winkler_distance_match_name_unidecode']>0.7)].reset_index(drop=True)
    
    return data


def get_match_variables(data,data_variables):

    data=data.merge(data_variables,on='id',how='left',suffixes=(None, '_id'))

    data=data.merge(data_variables,left_on='id_match',right_on='id',how='left',suffixes=(None, '_match_id'))
    
    return data
    
    

def create_candidates(data_position,measure_variables,n_nearest,data_features):
    
    data_position["longitude"]=np.radians(data_position["longitude"].values)
    
    data_position["latitude"]=np.radians(data_position["latitude"].values)
    
    distances,nearest_ids=nearest_neighbors(data_position[measure_variables],n_nearest=n_nearest)
    
    df_neighbors=[]
    
    
    for n_neighbor in range(1,n_nearest):
        
        df_temp=data_position[['id']].copy()
        
        df_temp['distance']=distances[:,n_neighbor]
        
        index_id_match=nearest_ids[:,n_neighbor]
        
        df_temp['id_match']=df_temp[['id']].loc[index_id_match].values
        
        df_temp['n_neighbor']=n_neighbor
        
        df_temp=get_match_variables(df_temp,data_features[['id','name']])
        
        df_temp=metrics_similiarity(df_temp,'name',distances_list=["jaccard","jaro_winkler"])
        
        df_temp=metrics_similiarity(df_temp,'name',distances_list=["jaccard","jaro_winkler"],make_unidecode=1,suffix="unidecode")
        
        print(len(df_temp))

        df_temp=filter_candidates(df_temp,n_neighbor)

        print(len(df_temp))
        print(df_temp["distance"].describe())

        df_neighbors.append(df_temp)
        
        print(n_neighbor)
        
        print('')
        
        
    df_neighbors=pd.concat(df_neighbors,ignore_index=True).reset_index(drop=True)
    
    df_neighbors=df_neighbors.drop(["id_match_id"],axis='columns')
    
    print(len(df_neighbors))
    
    df_temp=df_neighbors.copy()
    
    df_temp[["id","id_match"]]=df_temp[["id_match","id"]].copy()
    
    df_temp["n_neighbor"]=99
    
    df_neighbors=pd.concat([df_neighbors,df_temp],ignore_index=True)
    
    del df_temp ; gc.collect()
    
    df_neighbors=df_neighbors.drop_duplicates(subset=["id","id_match"],keep="first",ignore_index=True)
    
    print(len(df_neighbors))

    return df_neighbors


def get_perfect_jaccard_score(data):
    
    dict_predicted_matches=data.loc[data['target']==1].groupby('id')['id_match'].apply(list).to_dict()    
    
    scores = []
    for id_str, matches in tqdm(zip(df_real_matches['id'].to_numpy(), df_real_matches['id_match'].to_numpy())):
        
        if id_str in dict_predicted_matches:
            targets = dict_predicted_matches[id_str]
            targets.append(id_str)
            targets=set(targets)
        else :
            targets=set([id_str])
        preds = set(matches)
        score = len((targets & preds)) / len((targets | preds))
        scores.append(score)
    scores = np.array(scores)
    
    metric=scores.mean()
    
    print(metric)
    
    return metric


def get_tsvd_vectors(data,variable,n_components):
    
    variable_vectorizer = CountVectorizer(strip_accents="unicode",binary=True)
    variable_vectorizer = variable_vectorizer.fit(data[variable].fillna(""))
    variable_vectors=variable_vectorizer.transform(data[variable].fillna(""))

    tsvd_tranformer=TruncatedSVD(n_components=n_components)
    tsvd_tranformer=tsvd_tranformer.fit(variable_vectors)
    tsvd_vectors=tsvd_tranformer.transform(variable_vectors)
    
    return tsvd_vectors


def nearest_neighbors_arrays(data,n_nearest):
    
    knn = NearestNeighbors(n_neighbors = n_nearest)
    knn.fit(data,[1]*len(data))
    distances,nearest_ids = knn.kneighbors(data,return_distance = True)
    
    return distances,nearest_ids



def create_candidates_strings_vectors(data_id,data_position,n_nearest,data_features,variable="name"):
    
    distances,nearest_ids=nearest_neighbors_arrays(data_position,n_nearest=n_nearest)
    
    df_neighbors=[]
    
    
    for n_neighbor in range(0,n_nearest):
        
        df_temp=data_id[['id']].copy()
        
        df_temp['distance_vectors']=distances[:,n_neighbor]
        
        index_id_match=nearest_ids[:,n_neighbor]
        
        df_temp['id_match']=df_temp[['id']].loc[index_id_match].values
        
        df_temp['n_neighbor']=n_neighbor
        
        print(len(df_temp))
        
        df_temp=get_match_variables(df_temp,data_features)
        
        df_temp=metrics_similiarity(df_temp,variable,distances_list=["jaccard","jaro_winkler"])
        
        df_temp=metrics_similiarity(df_temp,variable,distances_list=["jaccard","jaro_winkler"],make_unidecode=1,suffix="unidecode")
                
        if variable == "name" :
        
            df_temp=filter_candidates_names(df_temp)
            
        elif variable=="address" :
            
            df_temp=filter_candidates_address(df_temp)
            
            
        
        print(len(df_temp))

        df_neighbors.append(df_temp)
        
        print(n_neighbor)
        
        print('')
        
        
    df_neighbors=pd.concat(df_neighbors,ignore_index=True).reset_index(drop=True)
    
    df_neighbors=df_neighbors.drop(["id_match_id"],axis='columns')
    
    print((df_neighbors["id"]==df_neighbors["id_match"]).mean())
    
    df_neighbors=df_neighbors.loc[df_neighbors["id"]!=df_neighbors["id_match"]].reset_index(drop=True)
    
    df_neighbors["n_neighbor"]=999
    
    df_neighbors["distance"]=999
    
    print(len(df_neighbors))

    return df_neighbors

  
def filter_candidates_names(data):
    
    data=data.loc[(data['jaccard_distance_match_name_']>0.95)|(data['jaro_winkler_distance_match_name_']>0.975)|(data['jaccard_distance_match_name_unidecode']>0.95)|(data['jaro_winkler_distance_match_name_unidecode']>0.975)].reset_index(drop=True)
    
    return data


def filter_candidates_address(data):
    
    data=data.loc[(data['jaccard_distance_match_address_']>0.95)|(data['jaro_winkler_distance_match_address_']>0.975)|(data['jaccard_distance_match_address_unidecode']>0.95)|(data['jaro_winkler_distance_match_address_unidecode']>0.975)].reset_index(drop=True)
    
    return data

In [10]:
df_train_strings=df_train.dropna(subset=["address"])[["id","name","address","latitude","longitude","point_of_interest"]].reset_index(drop=True)

name_vectors=get_tsvd_vectors(df_train_strings,"name",7)

address_vectors=get_tsvd_vectors(df_train_strings,"address",7)


neighbors_strings=create_candidates_strings_vectors(df_train_strings[["id"]],name_vectors,15,df_train_strings)

del name_vectors ; gc.collect()


address_neighbors_strings=create_candidates_strings_vectors(df_train_strings[["id"]],address_vectors,10,df_train_strings,variable='address')

del address_vectors
del df_train_strings; gc.collect()


print(len(neighbors_strings))

neighbors_strings=neighbors_strings.merge(address_neighbors_strings[["id","id_match"]],on=["id","id_match"])

print(len(neighbors_strings))

del address_neighbors_strings ; gc.collect()

neighbors_strings=neighbors_strings[['id','distance','id_match','n_neighbor','name','name_match_id','jaccard_distance_match_name_','jaro_winkler_distance_match_name_','jaccard_distance_match_name_unidecode','jaro_winkler_distance_match_name_unidecode']]

742191


742191it [00:11, 62945.46it/s]
742191it [00:19, 37931.47it/s]


709850
0

742191


742191it [00:12, 60926.10it/s]
742191it [00:19, 38018.24it/s]


362823
1

742191


...

100875
7

742191


742191it [00:12, 59466.12it/s]
742191it [00:20, 36426.88it/s]


93364
8

742191


742191it [00:12, 58699.49it/s]
742191it [00:20, 36628.53it/s]


87078
9

0.3350663974207576
1368825
2892789
74367


In [11]:
df_neighbors=create_candidates(df_train[['id','latitude','longitude']],measure_variables=["latitude","longitude"],n_nearest=61,data_features=df_train[["id","name"]])

print(len(df_neighbors))

df_neighbors=pd.concat([df_neighbors,neighbors_strings],ignore_index=True)

df_neighbors=df_neighbors.drop_duplicates(["id","id_match"]).reset_index(drop=True)

del neighbors_strings ; gc.collect()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
1138812it [00:18, 61026.69it/s]
1138812it [00:30, 37531.05it/s]


1138812
1138812
count    1.138812e+06
mean     4.345595e-05
std      6.851091e-04
min      0.000000e+00
25%      2.556714e-06
50%      6.575051e-06
75%      1.946656e-05
max      3.025971e-01
Name: distance, dtype: float64
1



1138812it [00:18, 61241.44it/s]
1138812it [00:30, 37718.08it/s]


1138812
230808
count    230808.000000
mean          0.000066
std           0.000602
min           0.000000
25%           0.000004
50%           0.000011
75%           0.000030
max           0.087485
Name: distance, dtype: float64
2



1138812it [00:18, 62155.45it/s]
1138812it [00:30, 37497.61it/s]


...

1138812it [00:18, 61109.97it/s]
1138812it [00:29, 38149.73it/s]


1138812
65872
count    65872.000000
mean         0.001002
std          0.005186
min          0.000000
25%          0.000052
50%          0.000150
75%          0.000513
max          0.706729
Name: distance, dtype: float64
59



1138812it [00:18, 61549.54it/s]
1138812it [00:29, 38065.03it/s]


1138812
65242
count    65242.000000
mean         0.001003
std          0.004740
min          0.000000
25%          0.000053
50%          0.000151
75%          0.000525
max          0.378813
Name: distance, dtype: float64
60

6428689
7852163
7852163


0

In [12]:
len(df_neighbors)

7860353

In [13]:
df_neighbors.columns

Index(['id', 'distance', 'id_match', 'n_neighbor', 'name', 'name_match_id',
       'jaccard_distance_match_name_', 'jaro_winkler_distance_match_name_',
       'jaccard_distance_match_name_unidecode',
       'jaro_winkler_distance_match_name_unidecode'],
      dtype='object')

In [14]:
df_neighbors.head(5)

,id,distance,id_match,n_neighbor,name,name_match_id,jaccard_distance_match_name_,jaro_winkler_distance_match_name_,jaccard_distance_match_name_unidecode,jaro_winkler_distance_match_name_unidecode
0,E_000001272c6c5d,0.000021,E_9e4a5adfe5b1c5,1,Café Stad Oudenaarde,Frituur De Groten Honger,0.000000,0.517929,0.000000,0.575000
1,E_000002eae2a589,0.000002,E_c12fb799a8a0e4,1,Carioca Manero,Claro,0.000000,0.725714,0.000000,0.725714
2,E_000007f24ebc95,0.000005,E_b664578fe0e8c2,1,ร้านตัดผมการาเกด,ร้านพี่นา,0.000000,0.773611,0.000000,0.792165
3,E_000008a8ba4f48,0.000003,E_40c486192f6105,1,Turkcell,ANker travel,0.000000,0.486111,0.000000,0.486111
4,E_00001d92066153,0.000002,E_7e0d8e9138dd56,1,Restaurante Casa Cofiño,Casa Cofiño,0.666667,0.356610,0.666667,0.429183


In [15]:
df_real_matches=df_train[['id','point_of_interest']].copy()

df_points_matches=df_train.groupby('point_of_interest')['id'].apply(list).reset_index()

df_points_matches=df_points_matches.rename(columns={'id':'id_match'})

df_real_matches=df_real_matches.merge(df_points_matches,on='point_of_interest',how='left')

df_real_matches=df_real_matches.drop('point_of_interest',axis='columns')

del df_points_matches ; gc.collect()

0

In [16]:

df_neighbors=get_match_variables(df_neighbors,df_train[['id',"point_of_interest","address",'city', 'state', 'zip','country', 'url', 'phone','categories','latitude','longitude']])

del df_train ; gc.collect()

df_neighbors=df_neighbors.drop(["id_match_id"],axis='columns')

df_neighbors['target']=(df_neighbors['point_of_interest']==df_neighbors['point_of_interest_match_id']).astype('int8')

print(df_neighbors['target'].mean())

print(df_neighbors['distance'].describe())

print("perfect jaccard score : ")

print(get_perfect_jaccard_score(data=df_neighbors[['id','id_match','target']]))

df_neighbors=create_n_words(df_neighbors)

df_neighbors=create_equal_indicator_features(df_neighbors,equal_variables=[ 'zip',
       'country', 'url', 'phone','categories'])

df_neighbors=metrics_similiarity(df_neighbors,'name',distances_list=["lcs","levenshtein","sequence_matcher","first_match_indicator","second_match_indicator","first_indicator","last_indicator"])

df_neighbors=metrics_similiarity(df_neighbors,'name',distances_list=["jaccard"],remove_vowels=1,suffix="vowels_unicode",make_unidecode=1)


df_neighbors=metrics_similiarity(df_neighbors,'name',distances_list=["lcs"],make_unidecode=1,suffix="unidecode")

df_neighbors=metrics_similiarity(df_neighbors,'address',distances_list=["jaccard","levenshtein","jaro_winkler","sequence_matcher"])

df_neighbors=metrics_similiarity(df_neighbors,'address',distances_list=["jaccard","jaro_winkler"],make_unidecode=1,suffix="unidecode")

df_neighbors=metrics_similiarity(df_neighbors,'categories',distances_list=["jaccard","levenshtein","jaro_winkler","sequence_matcher"])

df_neighbors=metrics_similiarity(df_neighbors,'city',distances_list=["jaccard"])

df_neighbors=metrics_similiarity(df_neighbors,'city',distances_list=["jaro_winkler"],make_unidecode=1,suffix="unidecode")

df_neighbors=metrics_similiarity(df_neighbors,'state',distances_list=["jaccard"])

df_neighbors=metrics_similiarity(df_neighbors,'state',distances_list=["jaro_winkler"],make_unidecode=1,suffix="unidecode")

df_neighbors=metrics_similiarity(df_neighbors,'zip',distances_list=["jaccard"])

df_neighbors=metrics_similiarity(df_neighbors,'zip',distances_list=["jaro_winkler"],make_unidecode=1,suffix="unidecode")


df_neighbors=metrics_similiarity(df_neighbors,'phone',distances_list=["jaro_winkler","sequence_matcher"])

df_neighbors=metrics_similiarity(df_neighbors,'name','url',distances_list=["jaro_winkler"],suffix='url')

df_neighbors=metrics_similiarity(df_neighbors,'name','url_match_id',distances_list=["jaro_winkler"],suffix='url_match_id')

df_neighbors=metrics_similiarity(df_neighbors,'name_match_id','url',distances_list=["jaro_winkler"],suffix='url')

df_neighbors=metrics_similiarity(df_neighbors,'name_match_id','url_match_id',distances_list=["jaro_winkler"],suffix='url_match_id')

df_neighbors=metrics_similiarity(df_neighbors,'name','address_match_id',distances_list=["jaccard"],suffix='name_address')

df_neighbors=metrics_similiarity(df_neighbors,'name_match_id','address',distances_list=["jaccard"],suffix='name_match_id_address')


df_neighbors=get_numbers_match(df_neighbors,'name')


df_neighbors=reduce_mem_usage(df_neighbors)


df_neighbors=get_categories_indicator(df_neighbors,"categories",count_vectorizer,10)

df_neighbors=get_categories_indicator(df_neighbors,"categories_match_id",count_vectorizer,10)

del count_vectorizer ; gc.collect()

df_neighbors=get_tsvd_lda_vectors(df_neighbors,"name",name_vectorizer,name_tsvd_transformer)

df_neighbors=get_tsvd_lda_vectors(df_neighbors,"name_match_id",name_vectorizer,name_tsvd_transformer)

del name_vectorizer
del name_tsvd_transformer ; gc.collect()

df_neighbors=get_tsvd_lda_vectors(df_neighbors,"categories",categories_vectorizer,categories_tsvd_transformer)

df_neighbors=get_tsvd_lda_vectors(df_neighbors,"categories_match_id",categories_vectorizer,categories_tsvd_transformer)

del categories_vectorizer
del categories_tsvd_transformer ; gc.collect()


0.11520373194435415
count    7.860353e+06
mean     1.041436e+00
std      3.222997e+01
min      0.000000e+00
25%      1.391294e-05
50%      5.143858e-05
75%      2.096843e-04
max      9.990000e+02
Name: distance, dtype: float64
perfect jaccard score : 


1138812it [00:03, 286109.95it/s]


0.9269549009883836
0.9269549009883836
zip
country
url
phone
categories


7860353it [13:43, 9547.86it/s] 
7860353it [04:24, 29701.26it/s]
7860353it [05:12, 25176.35it/s]
7860353it [10:58, 11940.74it/s]
7860353it [04:03, 32310.40it/s]
7860353it [10:03, 13032.05it/s]
7860353it [01:28, 88819.74it/s]
7860353it [02:04, 63070.41it/s]
7860353it [01:24, 93241.49it/s]
7860353it [01:56, 67274.84it/s]
7860353it [01:23, 94127.13it/s]
7860353it [01:32, 85214.35it/s]
7860353it [05:16, 24830.15it/s]
7860353it [01:52, 70103.84it/s]
7860353it [01:52, 70059.26it/s]
7860353it [01:53, 69331.39it/s]
7860353it [01:52, 69593.04it/s]
7860353it [01:55, 67860.06it/s]
7860353it [01:57, 67038.80it/s]
7860353it [01:22, 95002.88it/s]


Memory usage of dataframe is 5044.95 MB
Memory usage after optimization is: 2391.29 MB
Decreased by 52.6%


0

In [17]:
def entrena_lgb(data,test,features,categorical,target):

    kfold=GroupKFold(n_splits=5)


    i=1

    r=[]
    
    pred_test=np.zeros(len(test))

    importancias=pd.DataFrame()

    importancias['variable']=features
    
    list_models=[]
    
    
    cat_ind=[features.index(x) for x in categorical if x in features]
    
    dict_cat={}
    
    categorical_numerical = data[categorical].dropna().select_dtypes(include=np.number).columns.tolist()
    
    categorical_transform=[x for x in categorical if x not in categorical_numerical]
    
    for l in categorical_transform:
        le = preprocessing.LabelEncoder()
        le.fit(list(data[l].dropna())+list(test[l].dropna()))

        dict_cat[l]=le

        data.loc[~data[l].isnull(),l]=le.transform(data.loc[~data[l].isnull(),l])
        test.loc[~test[l].isnull(),l]=le.transform(test.loc[~test[l].isnull(),l])
        
        

    for train_index,test_index in kfold.split(data,data[target],data['point_of_interest']):

        lgb_data_train = lgb.Dataset(data.loc[train_index,features].values,data.loc[train_index,target].values)
        lgb_data_eval = lgb.Dataset(data.loc[test_index,features].values,data.loc[test_index,target].values, reference=lgb_data_train)

        params = {
            'task': 'train',
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': { 'auc'},
            "max_depth":-1,
            "num_leaves":256,
            'learning_rate': 0.1,
        "min_child_samples": 100,
            'feature_fraction': 0.9,
         "bagging_freq":1,
            'bagging_fraction': 0.9,
            "lambda_l1":10,
            "lambda_l2":10,
           # "scale_pos_weight":30,
           # 'min_data_per_group':500,

            'verbose': 1    
        }




        modelo = lgb.train(params,lgb_data_train,num_boost_round=13100,valid_sets=lgb_data_eval,early_stopping_rounds=50,verbose_eval=25,categorical_feature=cat_ind)

        importancias['gain_'+str(i)]=modelo.feature_importance(importance_type="gain")


        data.loc[test_index,'estimator']=modelo.predict(data.loc[test_index,features].values, num_iteration=modelo.best_iteration)
        
        pred_test=pred_test+modelo.predict(test[features].values, num_iteration=modelo.best_iteration)
        
        list_models.append(modelo)

        print ("Fold_"+str(i))
        a= (roc_auc_score(data.loc[test_index,target],data.loc[test_index,'estimator']))
        r.append(a)
        print (a)
        print ("")

        i=i+1
        
    for l in categorical_transform:

            data.loc[~data[l].isnull(),l]=dict_cat[l].inverse_transform(data.loc[~data[l].isnull(),l].astype(int))
            
            test.loc[~test[l].isnull(),l]=dict_cat[l].inverse_transform(test.loc[~test[l].isnull(),l].astype(int))
            
    var=[x for x in importancias.columns if 'gain_' in x]
    importancias["gain_avg"]=importancias[var].mean(axis=1)
    importancias=importancias.sort_values("gain_avg",ascending=False).reset_index(drop=True)
    
    pred_test=(pred_test/5)
    
    
    oof=(roc_auc_score(data[target],data['estimator']))
    
    print (oof)
    print ("mean: "+str(np.mean(np.array(r))))
    print ("std: "+str(np.std(np.array(r))))
    
    dict_resultados={}
    
    dict_resultados['importancias']=importancias
    
    dict_resultados['predicciones']=pred_test
    
    dict_resultados['modelos']=list_models
    
    
    
    return dict_resultados

In [18]:
variables_delete=['name', 'address', 'city', 'state', 'zip', 'country', 'url',
       'phone', 'categories', 
       'name_match_id', "latitude_match_id","longitude_match_id",
       'address_match_id', 'city_match_id', 'state_match_id', 'zip_match_id',
       'country_match_id', 'url_match_id', 'phone_match_id',
       'categories_match_id', 'point_of_interest_match_id',
         'numbers_name','numbers_name_match_id']
    
df_neighbors=df_neighbors.drop(variables_delete,axis="columns")

In [19]:
df_neighbors=reduce_mem_usage(df_neighbors)

Memory usage of dataframe is 3980.49 MB
Memory usage after optimization is: 1671.66 MB
Decreased by 58.0%


In [20]:
no_usar=['id',  'id_match', 'name', 'address', 'city', 'state', 'zip', 'country', 'url',
       'phone', 'categories', 'point_of_interest', 'id_match_id',
       'name_match_id', "latitude_match_id","longitude_match_id",
       'address_match_id', 'city_match_id', 'state_match_id', 'zip_match_id',
       'country_match_id', 'url_match_id', 'phone_match_id',
       'categories_match_id', 'point_of_interest_match_id',
         'numbers_name','numbers_name_match_id','numbers_address','numbers_address_match_id',"estimator","country_popular",
         'target']

features=[x for x in df_neighbors.columns if x not in no_usar]

categorical=[]

target='target'

In [21]:
features

['distance',
 'n_neighbor',
 'jaccard_distance_match_name_',
 'jaro_winkler_distance_match_name_',
 'jaccard_distance_match_name_unidecode',
 'jaro_winkler_distance_match_name_unidecode',
 'latitude',
 'longitude',
 'n_words_name',
 'n_characters_name',
 'n_words_name_match_id',
 'n_characters_name_match_id',
 'n_words_address',
 'n_characters_address',
 'n_words_address_match_id',
 'n_characters_address_match_id',
 'n_words_categories',
 'n_words_categories_match_id',
 'equal_zip',
 'equal_country',
 'equal_url',
 'equal_phone',
 'equal_categories',
 'first_match_indicator_match_name_',
 'second_match_indicator_match_name_',
 'first_indicator_match_name_',
 'last_indicator_match_name_',
 'levenshtein_distance_match_name_',
 'sequence_matcher_distance_match_name_',
 'lcs_distance_match_name_',
 'jaccard_distance_match_name_vowels_unicode',
 'lcs_distance_match_name_unidecode',
 'jaccard_distance_match_address_',
 'levenshtein_distance_match_address_',
 'jaro_winkler_distance_match_addr

In [22]:
df_neighbors['target'].mean()

0.11520373194435415

In [23]:
len(features)

105

In [24]:
df_test=df_neighbors.head(200)

In [25]:
dict_results=entrena_lgb(data=df_neighbors,test=df_test,features=features,categorical=categorical,target=target)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  _log_warning('categorical_feature in Dataset is overridden.\n'
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 718649, number of negative: 5569633
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.871817 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13787
[LightGBM] [Info] Number of data points in the train set: 6288282, number of used features: 105
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.114284 -> initscore=-2.047711
[LightGBM] [Info] Start training from score -2.047711
Training until validation scores don't improve for 50 rounds
[25]	valid_0's auc: 0.957041
[50]	valid_0's auc: 0.964791
[75]	valid_0's auc: 0.968281
[100]	valid_0's auc: 0.969865
[125]	valid_0's auc: 0.970934
[150]	valid_0's auc: 0.971571
[175]	valid_0's auc: 0.972129
[200]	valid_0's auc: 0.97239
[225]	valid_0's auc: 0.972499
[250]	valid_0's auc: 0.972676
[275]	valid_0's auc: 0.972818
[300]	valid_0's auc: 0.972895
[325]	valid_0's

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  _log_warning('categorical_feature in Dataset is overridden.\n'
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 729027, number of negative: 5559255
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.829441 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13785
[LightGBM] [Info] Number of data points in the train set: 6288282, number of used features: 105
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.115934 -> initscore=-2.031509
[LightGBM] [Info] Start training from score -2.031509
Training until validation scores don't improve for 50 rounds
[25]	valid_0's auc: 0.961919
[50]	valid_0's auc: 0.968539
[75]	valid_0's auc: 0.971516
[100]	valid_0's auc: 0.973438
[125]	valid_0's auc: 0.974604
[150]	valid_0's auc: 0.97542
[175]	valid_0's auc: 0.975968
[200]	valid_0's auc: 0.976378
[225]	valid_0's auc: 0.976719
[250]	valid_0's auc: 0.977007
[275]	valid_0's auc: 0.977223
[300]	valid_0's auc: 0.977397
[325]	valid_0's

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  _log_warning('categorical_feature in Dataset is overridden.\n'
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 725737, number of negative: 5562545
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.848965 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13770
[LightGBM] [Info] Number of data points in the train set: 6288282, number of used features: 105
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.115411 -> initscore=-2.036623
[LightGBM] [Info] Start training from score -2.036623
Training until validation scores don't improve for 50 rounds
[25]	valid_0's auc: 0.962066
[50]	valid_0's auc: 0.968882
[75]	valid_0's auc: 0.971781
[100]	valid_0's auc: 0.973438
[125]	valid_0's auc: 0.974574
[150]	valid_0's auc: 0.975206
[175]	valid_0's auc: 0.97573
[200]	valid_0's auc: 0.9761
[225]	valid_0's auc: 0.976368
[250]	valid_0's auc: 0.976653
[275]	valid_0's auc: 0.976832
[300]	valid_0's auc: 0.977033
[325]	valid_0's a

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  _log_warning('categorical_feature in Dataset is overridden.\n'
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 725365, number of negative: 5562918
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.837358 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13778
[LightGBM] [Info] Number of data points in the train set: 6288283, number of used features: 105
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.115352 -> initscore=-2.037203
[LightGBM] [Info] Start training from score -2.037203
Training until validation scores don't improve for 50 rounds
[25]	valid_0's auc: 0.95956
[50]	valid_0's auc: 0.966891
[75]	valid_0's auc: 0.970087
[100]	valid_0's auc: 0.972025
[125]	valid_0's auc: 0.973202
[150]	valid_0's auc: 0.973937
[175]	valid_0's auc: 0.974583
[200]	valid_0's auc: 0.974955
[225]	valid_0's auc: 0.975333
[250]	valid_0's auc: 0.975595
[275]	valid_0's auc: 0.97579
[300]	valid_0's auc: 0.976005
[325]	valid_0's 

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  _log_warning('categorical_feature in Dataset is overridden.\n'
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 723390, number of negative: 5564893
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.868375 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13759
[LightGBM] [Info] Number of data points in the train set: 6288283, number of used features: 105
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.115038 -> initscore=-2.040285
[LightGBM] [Info] Start training from score -2.040285
Training until validation scores don't improve for 50 rounds
[25]	valid_0's auc: 0.959621
[50]	valid_0's auc: 0.966215
[75]	valid_0's auc: 0.969115
[100]	valid_0's auc: 0.970602
[125]	valid_0's auc: 0.971651
[150]	valid_0's auc: 0.972323
[175]	valid_0's auc: 0.972669
[200]	valid_0's auc: 0.973022
[225]	valid_0's auc: 0.973254
[250]	valid_0's auc: 0.973534
[275]	valid_0's auc: 0.973767
[300]	valid_0's auc: 0.973863
[325]	valid_0'

In [26]:
dict_results['importancias']



,variable,gain_1,gain_2,gain_3,gain_4,gain_5,gain_avg
0,jaccard_distance_match_name_unidecode,2.284978e+06,2.016912e+06,2.005112e+06,2.274163e+06,2.251148e+06,2.166463e+06
1,jaccard_distance_match_name_vowels_unicode,1.999607e+06,2.164301e+06,2.217146e+06,1.967384e+06,2.016329e+06,2.072953e+06
2,n_neighbor,1.516226e+06,1.586145e+06,1.591575e+06,1.583429e+06,1.485779e+06,1.552631e+06
3,jaro_winkler_distance_match_name_unidecode,1.044517e+06,1.142161e+06,1.048885e+06,1.200674e+06,1.094354e+06,1.106118e+06
4,jaccard_distance_match_categories_,1.100839e+06,1.059117e+06,1.093707e+06,1.047284e+06,1.184951e+06,1.097180e+06
...,...,...,...,...,...,...,...
100,equal_categories,1.268451e+02,1.312344e+02,1.427668e+02,5.443633e+01,9.654420e+01,1.103654e+02
101,buildings_categories,1.043960e+01,1.457557e+02,7.914022e+01,2.044988e+02,2.157750e+01,9.228237e+01
102,offices_categories,0.000000e+00,3.630400e+01,2.552999e+02,2.889244e+01,4.516040e+00,6.500247e+01
103,buildings_categories_match_id,4.248200e+01,8.288935e+01,5.490738e+01,8.582721e+01,5.031821e+01,6.328483e+01


In [27]:
dict_results['importancias']["variable"].tolist()


['jaccard_distance_match_name_unidecode',
 'jaccard_distance_match_name_vowels_unicode',
 'n_neighbor',
 'jaro_winkler_distance_match_name_unidecode',
 'jaccard_distance_match_categories_',
 'distance',
 'latitude',
 'jaccard_distance_match_numbers_name_',
 'longitude',
 'sequence_matcher_distance_match_name_',
 'jaccard_distance_match_address_',
 'jaro_winkler_distance_match_name_',
 'second_match_indicator_match_name_',
 'first_match_indicator_match_name_',
 'jaccard_distance_match_name_',
 'categories_match_id_component_4',
 'categories_match_id_component_1',
 'categories_component_1',
 'categories_component_4',
 'sequence_matcher_distance_match_phone_',
 'jaro_winkler_distance_match_categories_',
 'lcs_distance_match_name_unidecode',
 'categories_component_0',
 'categories_match_id_component_0',
 'last_indicator_match_name_',
 'sequence_matcher_distance_match_address_',
 'levenshtein_distance_match_name_',
 'categories_match_id_component_2',
 'name_match_id_component_4',
 'categori

In [28]:
(df_neighbors['estimator']>0.1).mean()

0.1633994045814482

In [29]:
def adjust_results(df_results):

    df_results_adjusted=df_results.merge(df_results,left_on=["id","id_match"],right_on=["id_match","id"],suffixes=("_prob_1","_prob_2"))

    del df_results ; gc.collect()

    df_results_adjusted=df_results_adjusted.drop(['id_prob_2','id_match_prob_2'],axis='columns')

    df_results_adjusted=df_results_adjusted.rename(columns={"id_prob_1":"id","id_match_prob_1":"id_match"})

    df_results_adjusted["estimator"]=(df_results_adjusted["estimator_prob_1"]+df_results_adjusted["estimator_prob_2"])/2
    
    return df_results_adjusted


def get_transitive_matches(df_matches):
    
    df_extra_matches=df_matches.merge(df_matches,left_on="id_match",right_on="id",how="left")
        
    df_extra_matches=df_extra_matches.drop(['id_match_x','id_y'],axis="columns")
    
    df_extra_matches=df_extra_matches.rename(columns={"id_x":"id","id_match_y":"id_match"})
    
    df_extra_matches=pd.concat([df_matches,df_extra_matches],ignore_index=True)
    
    df_extra_matches=df_extra_matches.drop_duplicates(["id","id_match"],ignore_index=True)
    
    return df_extra_matches



def get_jaccard_score(data):
    
    #dict_predicted_matches=data.loc[condition].groupby('id')['id_match'].apply(list).to_dict()
    dict_predicted_matches=data.groupby('id')['id_match'].apply(list).to_dict()
    #df_real_matchs=data.loc[data['target']==1].groupby('id')['id_match'].apply(list).reset_index()
    
    scores = []
    for id_str, matches in tqdm(zip(df_real_matches['id'].to_numpy(), df_real_matches['id_match'].to_numpy())):
        
        if id_str in dict_predicted_matches:
            targets = dict_predicted_matches[id_str]
            if id_str not in targets :
                targets.append(id_str)
            targets=set(targets)
        else :
            targets=set([id_str])
        preds = set(matches)
        score = len((targets & preds)) / len((targets | preds))
        scores.append(score)
    scores = np.array(scores)
    
    metric=scores.mean()
    
    print(metric)
    
    return metric

In [30]:
df_results=df_neighbors[['id','id_match','estimator']].copy()

df_results_adjusted=adjust_results(df_results)


condition=df_results_adjusted["estimator"]>0.5

df_matches=df_results_adjusted.loc[condition,["id","id_match"]].reset_index(drop=True)


df_extra_matches=get_transitive_matches(df_matches)

In [31]:
    
get_jaccard_score(data=df_matches)



1138812it [00:04, 229160.56it/s]


0.8503682081826215


0.8503682081826215

In [32]:
get_jaccard_score(data=df_extra_matches)


1138812it [00:04, 263653.82it/s]


0.8526496851590587


0.8526496851590587

In [33]:
list_models=dict_results["modelos"]

In [34]:
with open('list_models_baseline_foursquare_v17.pkl', 'wb') as f:
    pickle.dump(list_models, f)

In [35]:
df_results.to_pickle("df_probas_train_foursquare.pkl")

In [36]:
df_results

,id,id_match,estimator
0,E_000001272c6c5d,E_9e4a5adfe5b1c5,0.011882
1,E_000002eae2a589,E_c12fb799a8a0e4,0.013435
2,E_000007f24ebc95,E_b664578fe0e8c2,0.089090
3,E_000008a8ba4f48,E_40c486192f6105,0.001762
4,E_00001d92066153,E_7e0d8e9138dd56,0.977308
...,...,...,...
7860348,E_f969a1f489442f,E_72f3ffecce328e,0.980889
7860349,E_faafa70d4d283f,E_e2bd3c766fa711,0.097540
7860350,E_fad0d7f7ec42ef,E_720482f0fd4487,0.004149
7860351,E_fca59242aa5ef3,E_77d2dee5525cf0,0.738150
